## 테스트

### 여기서는 그냥 간단하게, Training set, Test set 자체를 따로 선언

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2,2,2])

In [3]:
class SoftmaxClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
        
    def forward(self, x):
        return self.linear(x)

In [4]:
model = SoftmaxClassifier()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [5]:
def train(model, optimizer, x_train, y_train):
    total_epochs = 1000
    
    for epoch in range(total_epochs):
        prediction = model(x_train)
        cost = F.cross_entropy(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print('Epoch: {:4d}/{} Cost: {:.6f}'.format(
                epoch, total_epochs, cost.item()
            ))

In [6]:
def test(model, optimizer, x_test, y_train):
    prediction = model(x_test)
    predicted_classes = prediction.max(dim=1)[1] # Class값 출력
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)
    
    print('Accuracy: {}% Cost: {:.6f}'.format(
         correct_count / len(y_test) * 100, cost.item()
    ))

In [7]:
train(model, optimizer, x_train, y_train)

Epoch:    0/1000 Cost: 1.561535
Epoch:  100/1000 Cost: 0.707960
Epoch:  200/1000 Cost: 0.600865
Epoch:  300/1000 Cost: 0.541680
Epoch:  400/1000 Cost: 0.500026
Epoch:  500/1000 Cost: 0.467559
Epoch:  600/1000 Cost: 0.440921
Epoch:  700/1000 Cost: 0.418397
Epoch:  800/1000 Cost: 0.398962
Epoch:  900/1000 Cost: 0.381939


In [8]:
test(model, optimizer, x_test, y_test)

Accuracy: 100.0% Cost: 0.000684


## 전처리

#### X를 Standardize 했더니 학습 성능이 더 좋네

In [9]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

mu = x_train.mean(dim=0) 
sigma = x_train.std(dim=0) # Sample std (np.std라면 ddof=1인 version))

norm_x_train = (x_train-mu)/sigma

In [10]:
class MultivariateLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1)
    
    def forward(self, x):
        return self.linear(x)

In [11]:
model = MultivariateLinearRegression()
optimizer = optim.SGD(model.parameters(), lr=0.1)

def train(model, optimizer, x_train, y_train):
    total_epochs = 1000

    for epoch in range(total_epochs+1):

        prediction = model(x_train)
        cost = F.mse_loss(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print('Epoch {:4d}/{} Cost: {:.6f}'.format(
                epoch, total_epochs, cost.item()
            ))

In [12]:
train(model, optimizer, x_train, y_train)

Epoch    0/1000 Cost: 38852.039062
Epoch  100/1000 Cost: nan
Epoch  200/1000 Cost: nan
Epoch  300/1000 Cost: nan
Epoch  400/1000 Cost: nan
Epoch  500/1000 Cost: nan
Epoch  600/1000 Cost: nan
Epoch  700/1000 Cost: nan
Epoch  800/1000 Cost: nan
Epoch  900/1000 Cost: nan
Epoch 1000/1000 Cost: nan


In [13]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/1000 Cost: nan
Epoch  100/1000 Cost: nan
Epoch  200/1000 Cost: nan
Epoch  300/1000 Cost: nan
Epoch  400/1000 Cost: nan
Epoch  500/1000 Cost: nan
Epoch  600/1000 Cost: nan
Epoch  700/1000 Cost: nan
Epoch  800/1000 Cost: nan
Epoch  900/1000 Cost: nan
Epoch 1000/1000 Cost: nan
